In [1]:
import os
import xml.etree.ElementTree as ET
from utils.finder import find_leafes
from utils.dicom_to_nifti import seperate_labels

In [7]:
directory = r'C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\CT scans\CT scans Vaat (+)'
#directory = r'C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\CT scans\CT scans controle groep\temp'
annotation_dir = r'C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\Detail annotaties\Detail Vaat(+)'
#annotation_dir = r'C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\Detail annotaties\Detail Controle groep'
#boundingbox_dir = r"C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\Bounding boxes\BB Tumor(+), Vaat(+)"
output = r"C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\Processed_Data"

In [8]:
par_list = ['par', 'pan', '40']
pvp_list = ['ven', "pvp", "80", "port", "abdomen", "th abd ax 2 2", "tho+abd"]
art_list = ['art']
liver_list = ['laat']
ct_list = ['ct', 'tho', 'thx', 'buik']
patient_cases = {}

In [6]:
for case in os.listdir(directory):
    
    if os.path.isdir(os.path.join(directory, case)):
        case_number = case

        if int(case_number) != 1114:
            continue

        sectra = os.path.join(directory, case, "SECTRA", "CONTENT.XML")
        modalities = []
        if (os.path.exists(sectra)):

            root = ET.parse(sectra).getroot()
            print("==========================")
            print(case)
            print("==========================")

            for series in root.findall('patient/request/study/series'):
                for description in series.findall('series_data/description'):
                    #print(series)
                    id = series.get('id')
                    print(f"ID: {id}")
                    print(description.text)
                    phase = None

                    if description.text is not None:
                       
                        
                        if bool([ele for ele in par_list if(ele in description.text.lower())]):
                            print("PARENCHYMA")
                            phase = "par"
                        elif bool([ele for ele in pvp_list if(ele in description.text.lower())]):
                            print("PVP")
                            phase = "pvp"
                        elif bool([ele for ele in art_list if(ele in description.text.lower())]):
                            print("ART")
                            phase = "art"
                        elif bool([ele for ele in liver_list if(ele in description.text.lower())]):
                            print("liver")
                            phase = "liver"
                        else:
                            phase = "unknown"

                    else:
                        phase = "unknown"

                    modalities.append(phase)

                    if (phase == "unknown"):
                        print("Failed to find phase")
                        continue

                    high_data_dir = os.path.join(directory, case_number)
                    low_dir = find_leafes(high_data_dir)
                    data_dir = os.path.dirname(low_dir)

                    spec_data_dar = os.path.join(data_dir, id)
                    print(phase)
                    high_annotation_directory = os.path.join(annotation_dir, case_number, phase)
                   
                    spec_annotation_directory = find_leafes(high_annotation_directory)
                   
                    
                    if os.path.exists(spec_data_dar):
                        print(f"Data dir {spec_data_dar} found")
                    else:
                         print(f"Data dir {spec_data_dar} does not exist")
                         continue

                    try:
                        if os.path.exists(spec_annotation_directory): 
                            print(f"Annotation dir {spec_annotation_directory} found")
                    except:
                         print(f"Annotation dir {spec_annotation_directory} does not exist")
                         continue

                    
                    seperate_labels(spec_data_dar, spec_annotation_directory, output, case_number, phase)
                    
        else:

            print(f"{case} has an error")
            print(f"{sectra} does not exist")
            modalities.append("no info")
        patient_cases[case_number] = modalities


1097
ID: 00006C2B
ART 0.9/0.7
ART
art
Data dir C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\CT scans\CT scans Vaat (-)\1097\DICOM\00002F07\AA7FA599\AA34D62E\00006C2B found
Annotation dir None does not exist
ID: 00001133
PORT. 0.9/0.7
PVP
pvp
Data dir C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\CT scans\CT scans Vaat (-)\1097\DICOM\00002F07\AA7FA599\AA34D62E\00001133 found
Annotation dir C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\Detail annotaties\Detail Vaat(-)\1097\pvp\Patient 1097_1097\No study description_09-14-2015\Patient 1097 Pvp. S__(02-02-2021_16-08-59-1129) found
Reading: 1.2.840.113704.1.111.5228.1468569940.18.50.90512512
reader
itkSize3 ([512, 512, 593])
Image (000001C9DE528E80)
  RTTI typeinfo:   class itk::Image<short,3>
  Reference Count: 2
  Modified Time: 24525
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (000001C9D73A1240) 
  Source output name: Primary
  Release Data: 